In [34]:
import os
import openai
from google.colab import userdata
OPENAI_API_KEY=userdata.get("OPENAI_API_KEY")



In [31]:
#Prompt Summarization

from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [32]:
speech = """
All right. I’d like to thank you for leaving ‘crazy person’ out of the description. So, I thought — I was trying to think what’s the most useful thing that I – what I can say that can actually be helpful and useful to you in the future. And I thought, perhaps tell the story of how I sort of came to be here. How did these things happen? And maybe there are lessons there. I often find myself wondering, how did this happen.
When I was young, I didn’t really know what I was going to do when I got older. People kept asking me. But then eventually, I thought the idea of inventing things would be really cool.
And the reason I thought that was because I read a quote from Arthur C. Clark which said that “A sufficiently advanced technology is indistinguishable from magic.’ And that’s really true.
If you go back say, 300 years, the things we take for granted today, you’d be burned at stake for. Being able to fly. That’s crazy. Being able to see over long distances, being able to communicate, having effectively with the Internet as a group mind of sorts, and having access to all the world’s information instantly from almost anywhere on the earth. This stuff that really would be magic – that would be considered magic in times past.
In fact, I think it actually goes beyond that, because there are many things that we take for granted today that weren’t even imagined in times past, that weren’t even in the realm of magic. So, it actually goes beyond that. So I thought, well if I can do some of those things – basically if I can advance technology, then that is like magic and that would be really cool.
"""

In [37]:
chat_messages = [
    SystemMessage(content="You are an expert assistant with expertize in summarizing speeches"),
    HumanMessage(content=f"Please provide a short and concise summary of the following speech:\n TEXT: {speech}")
]

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo")

In [38]:
#total tokens

llm.get_num_tokens(speech)

357

In [40]:
#Get summary

print(llm.invoke(chat_messages).content)

In the speech, the speaker reflects on the idea of technology as a form of magic, drawing inspiration from a quote by Arthur C. Clarke. They share their journey of becoming an inventor and highlight the incredible advancements in technology that were once considered magical. The speaker emphasizes the importance of advancing technology to achieve seemingly magical feats and encourages the audience to consider the endless possibilities of innovation.


In [41]:
#Prompt Templates Text Summarization

from langchain.chains import LLMChain
from langchain import PromptTemplate

In [43]:
generic_template = """
Write a summary of the following speech:
Speech: `{speech}`
Translate the precise summary to {language}.
"""

In [44]:
prompt = PromptTemplate(
    input_variables = ["speech", "language"],
    template = generic_template
)

In [45]:
complete_prompt = prompt.format(speech=speech, language='French')

In [46]:
llm.get_num_tokens(complete_prompt)

376

In [52]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
summary = llm_chain.run({"speech": speech, "language":'french'})

In [53]:
summary

"Résumé: Le discours porte sur l'importance de l'innovation technologique et comment cela peut être considéré comme de la magie. Le conférencier raconte comment il a été inspiré par une citation d'Arthur C. Clarke et comment il a réalisé que inventer des choses pourrait être vraiment cool. Il souligne comment de nombreuses technologies avancées d'aujourd'hui étaient considérées comme de la magie dans le passé, et comment il vise à créer des avancées technologiques similaires."

In [56]:
# PDF Text Summarization

from PyPDF2 import PdfReader

In [57]:
pdfreader = PdfReader("/content/669683274-Elon-Musk-Speech.pdf")

In [59]:
from typing_extensions import Concatenate

In [60]:
text = ""
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    text += content

In [61]:
text

"Elon  Musk’s  Incredible  Speech  on  the\nEducation System | Eye Opening Video\non Education\nThere's no need even to have a college degree at all or even high school. I\nmean if somebody graduated from a great university that may be an indication\nthat they will be capable of great things, but it's not necessarily the case. You\nknow if you look at say people like Bill Gates, or Larry Ellison, Steve Jobs, these\nguys didn't graduate from college but if you had a chance to hire them of course\nthat  would  be  a  good  idea.  So  you  know  just  looking  just  for  evidence  of\nexceptional ability and if there's a track record of exceptional achievement then\nit's likely that that will continue into the future well first of all you don't need\ncollege learning learn stuff, everything is available basically for free you can learn\nanything you want for free.\nIt is not a question of learning there is a value that colleges have which is\nlike you know seeing whether somebody's can so

In [63]:
from langchain.docstore.document import Document

In [64]:
docs = [Document(page_content=text)]

In [65]:
docs

[Document(page_content="Elon  Musk’s  Incredible  Speech  on  the\nEducation System | Eye Opening Video\non Education\nThere's no need even to have a college degree at all or even high school. I\nmean if somebody graduated from a great university that may be an indication\nthat they will be capable of great things, but it's not necessarily the case. You\nknow if you look at say people like Bill Gates, or Larry Ellison, Steve Jobs, these\nguys didn't graduate from college but if you had a chance to hire them of course\nthat  would  be  a  good  idea.  So  you  know  just  looking  just  for  evidence  of\nexceptional ability and if there's a track record of exceptional achievement then\nit's likely that that will continue into the future well first of all you don't need\ncollege learning learn stuff, everything is available basically for free you can learn\nanything you want for free.\nIt is not a question of learning there is a value that colleges have which is\nlike you know seeing wh

In [67]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo", temperature=0)

In [68]:
from langchain.chains.summarize import load_summarize_chain

In [69]:
template = """
Write a concise and short summary of the following speech.
Speech: `{text}`
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template
)

In [71]:
chain = load_summarize_chain(
    llm,
    chain_type="stuff",
    prompt=prompt,
    verbose=False
)

output_summary = chain.run(docs)

In [72]:
output_summary

'Elon Musk questions the value of traditional education, stating that college degrees are not necessary for success. He emphasizes the importance of exceptional ability and problem-solving skills over formal education. Musk criticizes the current education system for lacking engagement and relevance, advocating for a more personalized and interactive approach to learning. He also discusses his own unconventional school, Ad Astra, which focuses on individual aptitudes and interests rather than traditional grade levels. Musk believes that education should be more like a video game, engaging and compelling, rather than a boring lecture. He suggests that the why of learning is crucial for motivation and retention of information. Ultimately, Musk argues that education should be tailored to individual needs and goals, rather than following a one-size-fits-all approach.'

In [73]:
#Summarizing Large Documents Using Map Reduce

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [74]:
pdfreader = PdfReader("/content/669683274-Elon-Musk-Speech.pdf")

text = ""
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    text += content

In [75]:
llm.get_num_tokens(text)

2398

In [77]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
chunks = text_splitter.create_documents([text])

In [78]:
chunks

[Document(page_content="Elon  Musk’s  Incredible  Speech  on  the\nEducation System | Eye Opening Video\non Education\nThere's no need even to have a college degree at all or even high school. I\nmean if somebody graduated from a great university that may be an indication\nthat they will be capable of great things, but it's not necessarily the case. You\nknow if you look at say people like Bill Gates, or Larry Ellison, Steve Jobs, these\nguys didn't graduate from college but if you had a chance to hire them of course\nthat  would  be  a  good  idea.  So  you  know  just  looking  just  for  evidence  of\nexceptional ability and if there's a track record of exceptional achievement then\nit's likely that that will continue into the future well first of all you don't need\ncollege learning learn stuff, everything is available basically for free you can learn\nanything you want for free.\nIt is not a question of learning there is a value that colleges have which is\nlike you know seeing wh

In [80]:
chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    verbose=False
)

summary = chain.run(chunks)

In [81]:
summary

'Elon Musk criticizes traditional education as outdated and unnecessary, advocating for individualized learning and problem-solving. He created Ad Astra school for his children, emphasizing the importance of exceptional ability over formal education. Musk believes education should be more engaging and relevant, tailored to individual interests and abilities rather than following a standardized curriculum. He suggests that solving problems is more engaging than teaching tools.'

In [82]:
# Map Reduce With Custom Prompts

chunk_prompt = """
Please summarize the below speech:
Speech: `{text}`
Summary:
"""
map_prompt_template = PromptTemplate(input_variables=["text"], template=chunk_prompt)

In [83]:
final_combine_prompt = """
Please provide a final summary of the entire speech with these important points.
Add a Generative Motivational Title.
Start the precise summary with an introduction and provide the summary in number points for the speech.
Speech: `{text}`
"""

final_combine_prompt_template = PromptTemplate(input_variables=["text"], template=final_combine_prompt)

In [84]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_combine_prompt_template,
    verbose=False
)

output = summary_chain.run(chunks)

In [85]:
output

'Title: "Unleashing Potential: A New Approach to Education"\n\nIntroduction:\nIn his speech, Elon Musk challenges the traditional education system and advocates for a more engaging and interactive approach to learning.\n\nSummary:\n1. Musk believes that traditional education is not necessary for success and that colleges are mainly for fun and proving one\'s ability to work hard.\n2. He created a school called Ad Astra for his children, where there are no grades and education is tailored to each child\'s interests and abilities.\n3. Musk advocates for a more engaging and interactive approach to education, comparing it to a video game.\n4. He believes that the why of learning is important and that education should focus on problem-solving rather than memorization.\n5. Musk suggests that university education may not be necessary for everyone, especially for those looking to start a company.\n6. He criticizes the assembly line approach to education and emphasizes the importance of explain

In [86]:
# RefineChain for Summarization

chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    verbose=True
)

output_summary = chain.run(chunks)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Elon  Musk’s  Incredible  Speech  on  the
Education System | Eye Opening Video
on Education
There's no need even to have a college degree at all or even high school. I
mean if somebody graduated from a great university that may be an indication
that they will be capable of great things, but it's not necessarily the case. You
know if you look at say people like Bill Gates, or Larry Ellison, Steve Jobs, these
guys didn't graduate from college but if you had a chance to hire them of course
that  would  be  a  good  idea.  So  you  know  just  looking  just  for  evidence  of
exceptional ability and if there's a track record of exceptional achievement then
it's likely that that will continue into the future well first of all you don't need
college learning learn stuff, everything is available basically for free you can learn
anything you wa